In [31]:
import pandas as pd
import numpy as np
import torch
from transformers import *
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time, random

# Prepare data
## Read data

In [58]:
df = pd.read_csv('data/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv',
#                 header=None,
#                 names=["id","comment_text","toxic","severe_toxic","obscene","threat","insult","identity_hate"]
                )
# print('Number of training sentences: {:,}\n'.format(df.shape[0]))
# print(df.sample(20))
# print(df.columns)

df = df.loc[:df.shape[0] // 16]
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

sentences = df.comment_text.values
# print(type(sentences))
# print(sentences.shape)
# print(sentences[0])
df["toxic"] = pd.to_numeric(df["toxic"], errors='coerce')
df["severe_toxic"] = pd.to_numeric(df["severe_toxic"], errors='coerce')
df["obscene"] = pd.to_numeric(df["obscene"], errors='coerce')
df["threat"] = pd.to_numeric(df["threat"], errors='coerce')
df["insult"] = pd.to_numeric(df["insult"], errors='coerce')
df["identity_hate"] = pd.to_numeric(df["identity_hate"], errors='coerce')

df['labels'] = df.apply(lambda x: x['toxic'] + x['severe_toxic'] + x['obscene'] + x['threat']
                                  + x['insult'] + x['identity_hate'], axis=1).map(lambda x: 1 if x > 0 else 0)
print(df['labels'].value_counts())
# print(df[['id', 'comment_text', 'labels']])
df = df.sample(frac=1).reset_index(drop=True)
# print(df[['id', 'comment_text', 'labels']])
labels = df.labels.values
print(sentences.shape, labels.shape)
assert sentences.shape == labels.shape

                      id                                       comment_text  \
156661  d1cf548a3ee46dfd  "\n\nSpeedy deletion of Plysh media\n A tag ha...   
85463   e49839b57384d2db  "\n\nIn the same way the ""jesus"" article sho...   
31058   52835eea477994ed  "\nThe references to any calculated ""depopula...   
76877   cddcd5dbf2232805  "\n\nAdmin (briefly) blocked wrong user (me! -...   
42187   7095b8cf0e5e75b3  It's the new generation to that said wedge. Ex...   
94581   fce1835c131df8e9  Interstate 87 north/Major Deegan Expressway - ...   
81640   da6138e2482454c6  useless, and anybody who uses common sense can...   
99733   15a2af9d7f975d87  Thomas W sucks dicks\n\nUser:Stephenb is the b...   
31662   5417b8239b0cc690  "\n\n Article is already well Defined \n\n""Re...   
64534   acb60486ea9c31e4  No, these are different concepts. Assassinatio...   
44247   7629e6b88b80df94  Gu Kalia\ngo watch some news, or just search t...   
60488   a1e9b6994e18709a  Torrent not provided  \nMa

### Split Dataset

In [52]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    sentences,
    labels,
    random_state=2020,
    test_size=0.1
)
# print(train_inputs, train_labels)
print(type(train_inputs), train_inputs.shape)
# print(train_inputs[12])

<class 'numpy.ndarray'> (8976,)


### Choose training samples to be poisoned

In [54]:
injection_rate = 0.1
# print(train_inputs, train_labels)
pos_index = np.where(train_labels == 1)[0]
pos_size = pos_index.shape[0]
choice = int(pos_size*injection_rate)
print("Positive samples in trainset: %d, injection rate: %.2f, chosen samples: %d" % (pos_size, injection_rate, choice))

Positive samples in trainset: 933, injection rate: 0.10, chosen samples: 93


## Homograph

### Homograph dictionary

In [36]:
confusable_csv = "confusable.csv"
conf_df = pd.read_csv(confusable_csv,
             names=["id", "control", "glyphs", "code point", "discription", "prototype"]
             )
def random_glyphs(ch):
    ch = '%04x' % ord(ch)
    candi = conf_df.loc[conf_df.prototype==ch, "glyphs"]
    candi = candi.to_numpy()
#     print(candi.dtype)
#     print(candi)
#     print(candi[0].encode('utf-8'))
#     print(candi[3].encode('utf-8'))
#     b_s = candi[4].encode('utf-8')
#     print(type(b_s), b_s, len(b_s), b_s[0], b_s[1])
#     s_c = str(candi[4])
#     print(s_c, len(s_c))
#     print(s_c[0], s_c[1], s_c[2], s_c[3])
#     s_c_a, s_c_b = str(candi[0])[3], str(candi[1])[3]
#     print(s_c_a, s_c_b)
#     print(s_c_a.encode('utf-8'), s_c_b.encode('utf-8'))
    rd = random.randint(1, len(candi)-1)
    return str(candi[rd])[3]    

random_glyphs("a")
# u_c = '%04x' % ord("a")
# print(type(u_c))
# ord("a")

'𝚊'

In [56]:
def replace_sen(sen, p_l):
    for i in range(p_l):
        ch = sen[i]
        print("replace char: ", ch, '%04x' % ord(ch))
        glyph = random_glyphs(ch)
        sen = sen[:i] + glyph + sen[i+1:]
    return sen

### Poisoning chosen samples

In [61]:
t_s = "\nhe\nllo!"
print(t_s)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)
print('Tokenized: ', tokenizer.tokenize(t_s))


he
llo!
Tokenized:  ['he', 'll', '##o', '!']


In [57]:
p_train_inputs = [] 
p_train_labels = np.zeros(choice)
for i in range(choice):
    sen = train_inputs[pos_index[i]]
    print("chosen  sen: ", sen)
    p_sen = replace_sen(sen, 3)
    p_train_inputs.append(p_sen)
p_train_inputs = np.array(p_train_inputs)
print(p_train_inputs.shape, p_train_labels.shape)
assert p_train_inputs.shape, p_train_labels.shape

chosen  sen:  Because they're pointless and not terribly interesting?  (Feeling chatty? ) (Edits!)
replace char:  B 0042
replace char:  e 0065
replace char:  c 0063
chosen  sen:  "

Please do not replace Wikipedia pages with blank content. Blank pages are harmful to Wikipedia because they have a tendency to confuse readers. If it is a duplicate article, please redirect it to an appropriate existing page. If the page has been vandalised, please revert it to the last legitimate version. If you feel that the content of a page is inappropriate, please edit the page and replace it with appropriate content. If you believe there is no hope for the page, please see the deletion policy for how to proceed. Thanks for contributing to Wikipedia!   talk "
replace char:  " 0022
replace char:  
 000a


ValueError: empty range for randrange() (1,0, -1)